In [1]:
# from pocketsphinx import LiveSpeech
# for phrase in LiveSpeech():
#     print(phrase)

In [2]:
# import speech_recognition as sr
# from SoundsLike.SoundsLike import Search

# recognizer = sr.Recognizer()

# while True:

#     try:

#         with sr.Microphone() as mic:

#             recognizer.adjust_for_ambient_noise(mic, duration=0.1)
#             audio = recognizer.listen(
#                 mic,
#                 phrase_time_limit=2,
#             )

#             text = recognizer.recognize_sphinx(
#                 audio,
#                 grammar=
#                 show_all=False 
#             )
#             text = text.lower()

#             print(f"{text}\n\r")
#             for word in text.split():
#                 # Search.perfectHomophones(word)
#                 print(word)
#                 print(Search.closeHomophones(word))

#     except :
#         print("Freak")
#         recognizer = sr.Recognizer()
#         continue

In [3]:
from pocketsphinx import get_model_path
print(get_model_path())

C:\Users\agrie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pocketsphinx\model


In [4]:
import os
from pocketsphinx import LiveSpeech
import random
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('', 12000))

speech = LiveSpeech(
     lm='./3427.lm', 
     hmm='./en-us/en-us',
	 dict = './3427.dic', 
     verbose=False, no_search=False, full_utt=False, buffer_size=1048, sampling_rate=16000
)


levenstein

In [5]:
# from Levenshtein import distance
# distance("selamat malam", "pagi selamat malam")

load datasets

In [6]:
import json
strdata = open('./datasets.json')
words = json.load(strdata)

get dataset data normalize

In [7]:
def getDataNormalize(items):
	# get all labels
	labels = list()
	indexLabels = list()
	labelIndex = 0

	patterns = list()
	minWord = 1
	maxWord = 1

	for item in items:
		labels.append(item['label'])
		for index in item['pattern']:
			indexLabels.append(labelIndex)
			patterns.append(index)
			# check min and max word
			wordCount = len(index.split())
			if minWord > wordCount:
				minWord = wordCount
			if maxWord <= wordCount:
				maxWord = wordCount
		# increase label index pointer
		labelIndex += 1
	result = {
		'labels': {
			'data': labels,
			'index': indexLabels
		},
		'patterns': {
			'data': patterns,
			'word': {
				'min': minWord,
				'max': maxWord
			}
		}
	}
	return result

In [8]:
data = getDataNormalize(words)
print(data)

{'labels': {'data': ['sambak', 'sari_sandu', 'nusa_fua_funi', 'ina_boi', 'ama_boi', 'deta_hitu', 'manek', 'tirosa'], 'index': [0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 7, 7, 7, 7]}, 'patterns': {'data': ['sambak', 'saambak', 'saree sandoo', 'sandoo', 'sari sandu', 'saari saandu', 'seri sendu', 'nusa fua funi', 'fua funi', 'nusa fua', 'noosa fooa fooni', 'fooa fooni', 'noosa fooa', 'ina bo ee', 'ina boi', 'ina', 'eena boee', 'aama bo ee', 'ama boi', 'ama', 'ama bo ee', 'deta heetoo', 'deta hitu', 'detaa heetow', 'detaa', 'heetow', 'maanek', 'manek', 'tirosa', 'teerosaa', 'tee rosaa', 'rosaa'], 'word': {'min': 1, 'max': 3}}}


tokenizing

In [9]:
import math
import pandas as pd

def tokenizing(words, wordOptions):
	wordsList = words.strip().split()
	wordsCount = len(wordsList)
	result = list()
	# divide by min and max word section

	for wordSection in range(wordOptions['min'], wordOptions['max'] + 1):
		# split words
		sectionAvailable = wordsCount - (wordSection - 1)

		startWord = 0

		for section in range(sectionAvailable) :
			startWord = section
			endWord = wordSection + startWord
			if endWord >= wordsCount:
				endWord = wordsCount
			wordJoined = " ".join(wordsList[startWord:endWord])
			# print(wordSection, section, startWord, sectionAvailable, wordJoined)
			result.append(wordJoined)
	return result



In [10]:
print('aana faeto ndana ta neeta camba'.split(" ")[1:1])

[]


In [11]:

def parseVoice(word):
    print(word)
    tokenized = tokenizing(words=word, wordOptions=data['patterns']['word'])
    # check the tokenized match words
    # reverse tokenized
    # print(tokenized)
    tokenized.reverse()
    # print(tokenized)
    patternMatched = None
    for token in tokenized:
        # print(tokenized)
        for index, pattern in enumerate(data['patterns']['data']):
            # print(pattern, "==", token, pattern == token)
            if pattern == token:
                patternMatched = index
            if patternMatched is not None:
                break
        if(patternMatched is not None):
            break
    label = None
    if(patternMatched is not None):
        labelIndex = data['labels']['index'][patternMatched]
        label = data['labels']['data'][labelIndex]
    return {
        'label': label,
        'pattern': patternMatched
    }


In [12]:

while True:
    message, address = server_socket.recvfrom(1024)
    print(message, address)
    for phrase in speech:
        print(phrase)
        # print(message)
        word = str(phrase)
        # print(word)
        parsedVoice = parseVoice(word.lower())
        try:
            if(parsedVoice['pattern'] is not None):
                server_socket.sendto(str.encode(
                    parsedVoice['label']
                    # word
                ), address)
        except:
            "error"
        

b'siddheshrane' ('127.0.0.1', 65321)
EENA BOEE
eena boee
AAMA BO HEETOO
aama bo heetoo
DETAA HEETOW
detaa heetow




DETAA HEETOW
detaa heetow
DETA AMA BOEE SARI EE INA INA EE FUNI INA BO SARI DETA HITU SAMBAK DETA DETAA HEETOW DETA HITU FUA DETA DETA HEETOO TEE
deta ama boee sari ee ina ina ee funi ina bo sari deta hitu sambak deta detaa heetow deta hitu fua deta deta heetoo tee
HEETOO EE DETAA HEETOW BO HITU HITU SAMBAK DETA FOONI FUA AMA FOONI HITU INA MAANEK DETA FUA BO AMA AMA BOI AMA DETA HITU DETA HEETOO DETA BO DETA HEETOO HEETOO DETA HITU FUNI DETA HEETOO MAANEK DETA BO EE AMA INA DETA DETA HEETOO FOOA FOONI MAANEK BO EE HEETOO TEE DETA TEE EE FUNI
heetoo ee detaa heetow bo hitu hitu sambak deta fooni fua ama fooni hitu ina maanek deta fua bo ama ama boi ama deta hitu deta heetoo deta bo deta heetoo heetoo deta hitu funi deta heetoo maanek deta bo ee ama ina deta deta heetoo fooa fooni maanek bo ee heetoo tee deta tee ee funi
DETA AMA INA FUA DETA DETA DETA HITU MANEK INA BO E